Ecommerce Database

The goal of the project is to create a relational sample database using Python. Then use SQL Server to update and explore the relational database. Lastly to create a dashboard in Tableau and Excel to explore KPI's


In [46]:
#imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import string
import numpy as np
import datetime
from datetime import timedelta


In [47]:
#Create tables for the relational database
product = pd.DataFrame(columns = ['sku','name','price','quantity','cogs'])
orders = pd.DataFrame(columns = ['order_id','created_at'])
ordered_items = pd.DataFrame(columns = ['order_id','sku','quantity_ordered'])

In [48]:
#Downloading products and prices from Lego.com for starwars products

URL = "https://www.lego.com/en-us/themes/star-wars"
page = requests.get(URL)
product_name = []
product_price = []
price = []

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="blt4e128e4a78935e4b")
for x in range(1,8):
  URL = "https://www.lego.com/en-us/themes/star-wars?page=" + str(x)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  results = soup.find(id="blt4e128e4a78935e4b")
  Lego_sets = results.find_all("div", class_="ProductLeafSharedstyles__Wrapper-sc-1yg7ucv-0 dWswSG")
  for lego_set in Lego_sets:
    t = lego_set.find("h2", class_="Text__BaseText-sc-13i1y3k-0 iSNFVS ProductLeafSharedstyles__Title-sc-1yg7ucv-9 zimKm")
    n = str(t.find(class_="Markup__StyledMarkup-nc8x20-0 epIXnJ"))
    m=n.split('>')[1].split('<')[0]
    product_name.append(m)

    p = lego_set.find("span", class_="Text__BaseText-sc-13i1y3k-0 eTDhBg ProductPricestyles__StyledText-vmt0i4-0 chARQH")
    if p == None:
      p = lego_set.find("span",class_="Text__BaseText-sc-13i1y3k-0 fOBIfn")
    
    q = str(p).split('$')[1].split('<')[0]
    product_price.append(q)



    


In [49]:
#Create the product dataframe

#temp list to assign values
prices = []
sku_list = []
cog = []
quantities = []

#convert price string to float - better for analsys in sql and tableau
for i in product_price:
  prices.append(round(float(i),2))

#random string to make up skus
characters = string.ascii_letters + string.digits

for x in range(len(product_name)):
  sku_list.append(''.join(random.choice(characters) for i in range(5)))
  
#calculate the cost of goods sold
for i in prices:
  cog.append(i/2.00)

#assign quantities a normal distribution (how many of each unit in warehouse)
quantities = np.round(np.random.normal(35, 3, len(product_name)), 0)

product = product.assign(quantity = quantities)
product = product.assign(sku = sku_list)
product = product.assign(name = product_name)
product = product.assign(price = prices)
product = product.assign(cogs = cog)

In [50]:
product

,sku,name,price,quantity,cogs
0,edbWy,Executor Super Star Destroyer™,69.99,36.0,34.995
1,3PmVz,Spider Tank,49.99,36.0,24.995
2,7J04T,Captain Rex™ Helmet,69.99,37.0,34.995
3,G7fnV,The Razor Crest™,599.99,37.0,299.995
4,IF968,AT-TE™ Walker,139.99,40.0,69.995
...,...,...,...,...,...
94,csft6,Millennium Falcon™ Microfighter,7.99,35.0,3.995
95,GfWvN,Darth Vader™ Key Chain,5.99,35.0,2.995
96,9SIK4,Scout Trooper™ Key Chain,5.99,36.0,2.995
97,at5wn,Fennec Shand™ Key Chain,5.99,32.0,2.995


In [51]:
#Order Table
orderids = []
#sting for unique order ids
ids = string.digits

for x in range(500):
  orderids.append(''.join(random.choice(ids) for i in range(7)))

orders = orders.assign(order_id = orderids)
#check for duplicates
orders[orders.duplicated(['order_id'],keep=False)]

#Create order dates using datetime functions
date = []
for y in range(500):
  start_date = datetime.date(2021,1,1)
  end_date = datetime.date(2022,1,1)
  rand_date = start_date + (end_date - start_date) * random.random()
  date.append(rand_date)

orders = orders.assign(created_at = date)



In [52]:
orders

,order_id,created_at
0,7639468,2021-03-13
1,5792515,2021-08-14
2,1442754,2021-03-28
3,4913410,2021-03-30
4,5465905,2021-05-22
...,...,...
495,1921077,2021-12-08
496,8536383,2021-11-20
497,9838185,2021-10-18
498,0225187,2021-04-08


In [53]:
#ordered_items table (line items of what each order contains)
ordered_skus = []
quantity_order = []
orderid = []
possible_orders = [1, 2, 3, 4]
value = []
for i in range(1000):
  ordered_skus.append(random.choice(sku_list))
  #assign weights to each number, i.e. most likely only 1 item ordered at a time
  value.append(random.choices(possible_orders, weights=(65, 25, 5, 5,), k=1))
  orderid.append(random.choice(orderids))



ordered_items = ordered_items.assign(sku = ordered_skus)
quantity_order = [int(x[0]) for x in value]
ordered_items = ordered_items.assign(quantity_ordered = quantity_order)
ordered_items = ordered_items.assign(order_id = orderid)




In [54]:
ordered_items

,order_id,sku,quantity_ordered
0,5572901,VH7g4,1
1,7692810,EYuPh,1
2,2201601,VH7g4,1
3,2582187,F2FMD,1
4,7639468,6uQgs,2
...,...,...,...
995,6015554,l8R8p,1
996,5982038,6s0in,1
997,3209289,l8R8p,1
998,9177637,On85X,2


In [56]:
#calculating the order total amount using ordered_items
#group by and merge tables 

temp = product.merge(ordered_items, on='sku')
temp['amount_ordered'] = temp['price'] * temp['quantity_ordered']
temp1 = pd.DataFrame(temp.groupby(['order_id'])['amount_ordered'].agg('sum'))

orders.merge(temp1,on='order_id')

,order_id,created_at,amount_ordered
0,7639468,2021-03-13,23.96
1,5792515,2021-08-14,213.93
2,1442754,2021-03-28,239.96
3,4913410,2021-03-30,133.99
4,0510900,2021-12-01,145.96
...,...,...,...
430,1919025,2021-01-07,449.96
431,6898222,2021-03-22,105.97
432,9838185,2021-10-18,171.94
433,0225187,2021-04-08,92.96


In [58]:
#Download data for SQL Server and Tableau
from google.colab import files
orders.to_csv('orders.csv', encoding = 'utf-8-sig') 
files.download('orders.csv')
ordered_items.to_csv('ordereditems.csv', encoding = 'utf-8-sig') 
files.download('ordereditems.csv')
product.to_csv('product.csv', encoding = 'utf-8-sig') 
files.download('product.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>